In [1]:
''' This is a sample for celestial navigation for a stationary observer 
    © August Linnman, 2024, email: august@linnman.net
    MIT License (see LICENSE file)

    Jupyter notebook for ordinary three-fix celestial navigation (stationary).
'''

# Importing Python libraries
from time import time
from starfix import Sight, SightCollection, get_representation, get_google_map_string,\
                    IntersectError

THE_POS = LatLonGeodetic (42, -88)

NameError: name 'LatLonGeodetic' is not defined

In [7]:
# SIGHT NR 1. 

a = Sight (   object_name          = "Sun",
              set_time             = "2024-05-05 15:55:18+00:00",
              gha_time_0           = "45:50.4",
              gha_time_1           = "60:50.4",
              decl_time_0          = "16:30.6",
              decl_time_1          = "16:31.3",
              measured_alt         = "55:8:1.8",
              estimated_position   = THE_POS
              )




In [8]:
# SIGHT NR 2.

b = Sight (   object_name          = "Sun",
              set_time             = "2024-05-05 23:01:19+00:00",
              gha_time_0           = "165:50.8",
              gha_time_1           = "180:50.8",
              decl_time_0          = "16:36.2",
              decl_time_1          = "16:36.9",
              measured_alt         = "19:28:18",
              estimated_position   = THE_POS
              )

In [9]:
# SIGHT NR 3. 

c = Sight (   object_name          = "Vega",
              set_time             = "2024-05-06 04:04:13+00:00",
              gha_time_0           = "284:30.4",
              gha_time_1           = "299:32.9",
              decl_time_0          = "38:48.1",
              measured_alt         = "30:16:24.6",
              sha_diff             = "80:33.4",
              estimated_position   = THE_POS
              )

In [10]:
# SIGHT REDUCTION.

collection = SightCollection ([a, b, c])
try:
    intersections, _, _ = collection.get_intersections (return_geodetic=True)
    print (get_representation(intersections,1))
    print ("MD = " + collection.get_map_developers_string(geodetic=True))
    print ("GM = " + get_google_map_string(intersections,4))    
except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve))
    print ("Check the circles! " + collection.get_map_developers_string(geodetic=True))    


(N 41°,51.3′;W 87°,38.6′)
MD = https://www.mapdevelopers.com/draw-circle-tool.php?circles=%5B%5B3879316%2C16.5208%2C-59.665%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B7849750%2C16.6036%2C-166.1758%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B6646299%2C38.8017%2C-6.1204%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%5D
GM = 41.8555,-87.6429
